# Deepfake Detector LoRA Fine-Tuning on Celeb-DF-v2

**Instructions:**
1. Runtime → Change runtime type → T4 GPU
2. Run all cells in order
3. Wait 3-6 hours for training
4. Download trained model from last cell

**Estimated time:** 3-6 hours on T4 GPU

In [ ]:
# Step 1: Install dependencies
!pip install -q transformers peft accelerate scikit-learn pillow tqdm datasets

In [ ]:
# Step 2: Upload the prepared dataset
# Option A: Upload celeb_df_processed.zip from your computer
from google.colab import files
import zipfile
import os

print("Please upload celeb_df_processed.zip (the folder with train/val data)")
print("This file should be on your local machine at: /mnt/e/szakdoga/Deepfake/")
print("\nZip it first with: cd /mnt/e/szakdoga/Deepfake && zip -r celeb_df_processed.zip celeb_df_processed/")
print("\nThen upload here:")

uploaded = files.upload()

# Extract
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f'\nExtracting {filename}...')
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print('Extraction complete!')

# Verify
!ls -lh celeb_df_processed/train/real/ | head -5
!ls -lh celeb_df_processed/train/fake/ | head -5

In [ ]:
# Step 3: Import libraries and check GPU
import torch
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import json

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  WARNING: No GPU detected! Training will be VERY slow.")
    print("Go to Runtime → Change runtime type → T4 GPU")

In [ ]:
# Step 4: Define dataset class
class DeepfakeDataset(Dataset):
    def __init__(self, data_dir, split='train', processor=None):
        self.data_dir = Path(data_dir) / split
        self.processor = processor
        self.images = []
        self.labels = []

        # Load real images (label=1)
        real_dir = self.data_dir / 'real'
        for img_path in real_dir.glob('*.jpg'):
            self.images.append(str(img_path))
            self.labels.append(1)

        # Load fake images (label=0)
        fake_dir = self.data_dir / 'fake'
        for img_path in fake_dir.glob('*.jpg'):
            self.images.append(str(img_path))
            self.labels.append(0)

        print(f"Loaded {split} set: {len(self.images)} images")
        print(f"  Real: {sum(self.labels)} images")
        print(f"  Fake: {len(self.labels) - sum(self.labels)} images")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        label = self.labels[idx]

        if self.processor:
            encoding = self.processor(image, return_tensors='pt')
            encoding = {k: v.squeeze(0) for k, v in encoding.items()}
        else:
            encoding = {'pixel_values': image}

        encoding['labels'] = torch.tensor(label, dtype=torch.long)
        return encoding

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary', zero_division=0
    )
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# Step 5: Load base model and configure LoRA
BASE_MODEL = "prithivMLmods/deepfake-detector-model-v1"
DATA_DIR = "celeb_df_processed"
OUTPUT_DIR = "celeb_df_finetuned"

print("Loading base model...")
processor = AutoImageProcessor.from_pretrained(BASE_MODEL)
model = AutoModelForImageClassification.from_pretrained(
    BASE_MODEL,
    num_labels=2,
    ignore_mismatched_sizes=True
)

model.config.id2label = {0: "fake", 1: "real"}
model.config.label2id = {"fake": 0, "real": 1}

total_params = sum(p.numel() for p in model.parameters()) / 1e6
print(f"Base model loaded: {total_params:.2f}M parameters")

# Configure LoRA
print("\nConfiguring LoRA...")
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
)

model = get_peft_model(model, lora_config)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6

print(f"LoRA Configuration:")
print(f"  Trainable parameters: {trainable_params:.2f}M ({trainable_params/total_params*100:.1f}% of total)")
print(f"  Memory savings: ~{(1 - trainable_params/total_params)*100:.0f}%")
model.print_trainable_parameters()

In [ ]:
# Step 6: Load datasets
print("Loading datasets...")
train_dataset = DeepfakeDataset(DATA_DIR, split='train', processor=processor)
val_dataset = DeepfakeDataset(DATA_DIR, split='val', processor=processor)

In [ ]:
# Step 7: Configure training
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
    fp16=True,  # Use mixed precision on GPU
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print("Training configuration complete!")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Learning rate: {training_args.learning_rate}")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"FP16 (mixed precision): {training_args.fp16}")

In [ ]:
# Step 8: START TRAINING (this will take 3-6 hours)
print("="*60)
print("STARTING TRAINING - This will take 3-6 hours")
print("You can close this tab and come back later")
print("="*60)

train_result = trainer.train()

print("\nTraining complete!")

In [ ]:
# Step 9: Save model and run final evaluation
print("Saving model...")
model.save_pretrained(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)

print("Running final evaluation...")
eval_metrics = trainer.evaluate()

# Save config
config = {
    "base_model": BASE_MODEL,
    "fine_tuning_method": "LoRA",
    "lora_r": 16,
    "lora_alpha": 32,
    "dataset": "Celeb-DF-v2",
    "train_samples": len(train_dataset),
    "val_samples": len(val_dataset),
    "final_metrics": eval_metrics
}

with open(f"{OUTPUT_DIR}/training_config.json", 'w') as f:
    json.dump(config, f, indent=2)

print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60)
print(f"\nFinal Metrics:")
print(f"  Accuracy: {eval_metrics['eval_accuracy']:.4f}")
print(f"  F1 Score: {eval_metrics['eval_f1']:.4f}")
print(f"  Precision: {eval_metrics['eval_precision']:.4f}")
print(f"  Recall: {eval_metrics['eval_recall']:.4f}")
print(f"\nModel saved to: {OUTPUT_DIR}/")

In [ ]:
# Step 10: Download the trained model
!zip -r celeb_df_finetuned.zip celeb_df_finetuned/

print("\n📥 DOWNLOAD YOUR TRAINED MODEL:")
from google.colab import files
files.download('celeb_df_finetuned.zip')

print("\n✅ Download complete!")
print("\nNext steps:")
print("1. Extract celeb_df_finetuned.zip on your local machine")
print("2. Place it in: /mnt/e/szakdoga/Deepfake/models/")
print("3. Update detector.py to load this model")
print("4. Test on your Celeb-DF-v2 videos!")